<a href="https://colab.research.google.com/github/jvrupp/Airbnb/blob/main/Analise_Airbnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sistema que consegue definir um perfil dos anuncios melhor e pior posicionados no site airbnb , trzendo entao uma analise de que tipo de casa seria mais interessante para aumentar a conversao em locaçoes.


In [ ]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

data = pd.read_csv(r'C:\Users\jvrup\Airbnb\Airbnb\BancoGeral2.csv',decimal=",")
   

#estratificando precos############################################
#bol = data['lugar']=='Ilhéus - BA'
#data = data[bol] 

data=data.replace('Estúdio', 1)
data['quarto']=pd.to_numeric(data['quarto'])

#removendo os pontos e tratando o grafico##########################################
data['preco']=data['preco'].astype(str).str.replace('.', '').astype(int)


bol=data[(data['banheiros'] != 'Estacionamento')]
bol=data[(data['banheiros'] != 'Ar-condicionado')]
bol = bol[bol['banheiros']!='Lavabo']
bol = bol[bol['banheiros']!='Wi-Fi']
data =bol[:]
data['banheiros']=pd.to_numeric(data['banheiros'])

data['P/h']= data['preco']/data['Hospedes']



#clusterizacao#############################################################

#Dividindo os DataFrames por cidade
Barra_Grande = data[data['lugar']=='Barra Grande - BA'].iloc[-18:,:]
Itacare = data[data['lugar']=='Itacare - BA'].iloc[-18:,:]
Ilheus = data[data['lugar']=='Ilhéus - BA'].iloc[-18:,:]
Arraial_djuda = data[data['lugar']=='Arraial djuda - BA'].iloc[-18:,:]
Porto_Seguro = data[data['lugar']=='Porto Seguro - BA'].iloc[-18:,:]
Caraiva = data[data['lugar']=='Caraiva - BA'].iloc[-18:,:]
Morro_de_sao_paulo = data[data['lugar']=='Morro de são paulo - BA'].iloc[-18:,:]

lista_lugares_pagina0 = [Barra_Grande,Itacare,Ilheus,Arraial_djuda,Porto_Seguro,Caraiva,Morro_de_sao_paulo]

###########################################################################



onlyFrstPg= data[data['pagina']==1].iloc[:126,:]

onlyFrstPgVar= data[data['pagina']==1].describe()
onlyOthersPg=pd.DataFrame()
for i in lista_lugares_pagina0:
    onlyOthersPg = onlyOthersPg.append(i, ignore_index=True)
onlyOthersPgVar= data[data['pagina']!=1].iloc[-128:,:].describe()

bigdata = onlyFrstPg.append(onlyOthersPg, ignore_index=True)

X = bigdata.iloc[:,[5,6,7,8,9,10]]
Y = bigdata.iloc[:,2]

############################################################################
scaler=StandardScaler()
scaler.fit(bigdata.iloc[:,5:])
scaled=scaler.transform(bigdata.iloc[:,5:])

def calculate_wcss(data):
        wcss = []
        for n in range(1, 11):
            kmeans = KMeans(n_clusters=n)
            kmeans.fit(X=data)
            wcss.append(kmeans.inertia_)
    
        return wcss
    
def calculate_cluster(data):
    kmeans = KMeans(n_clusters=4)
    predict=kmeans.fit_predict(X=data)
    
    return predict
    
#measureCluster=pd.DataFrame(calculate_wcss(scaled))
#measureCluster.plot()
cluster= calculate_cluster(scaled)
bigdata['grupos']=cluster
############################################################################

pag1Grupo0 = (bigdata['grupos'] ==1) & (bigdata['pagina']==1)
bol2=bigdata[pag1Grupo0]
describeBol2 = bol2.describe()


X_Tr,X_Te,Y_Tr,Y_te=train_test_split(X, Y, test_size=0.3333,random_state=42)  



clf = RandomForestClassifier(n_estimators=1000,random_state=42).fit(X_Tr, Y_Tr)
Fi=clf.feature_importances_
Fi=pd.DataFrame(Fi)
y_pred=clf.predict(X_Te)
Fi.plot.bar()
print(classification_report(Y_te,y_pred)) 
print(Y_Tr.value_counts())


itacare_3=(bol['quarto']==3)
itacare_3 = bol[itacare_3]


"""
para um cenario geral da regiao 
Progredir a mescla de variaveis

Primeiro obejetivo 184 reais por dia
4 hospedes
1 quarto
2 camas
1 banheiro
4.89++ de nota
66 comentarios
Preço/Hospede  = 60 reais